# Scaling Memory-Augmented Neural Networks with Sparse Reads and Writes

* 싸이그래머 / ReMind : 파트 1 - 딥마인드 논문 리뷰 [1]
* 김무성

# Contents
* Abstract
* 1 Introduction
* 2 Background
    - 2.1 Attention and content-based addressing
    - 2.2 Memory Networks
    - 2.3 Neural Turing Machine
* 3 Architecture
    - 3.1 Read
    - 3.2 Write
    - 3.3 Controller
    - 3.4 Efficient backpropagation through time
    - 3.5 Approximate nearest neighbors
* 4 Results
    - 4.1 Speed and memory benchmarks
    - 4.2 Learning with sparse memory access
    - 4.3 Scaling with a curriculum
    - 4.4 Question answering on the Babi tasks
    - 4.5 Learning on real world data
* 5 Discussion

#### 참고
* [1] Scaling Memory-Augmented Neural Networks with Sparse Reads and Writes - https://arxiv.org/abs/1610.09027
* [2] Daniel Shank, Data Scientist, Talla at MLconf SF 2016 - http://www.slideshare.net/SessionsEvents/daniel-shank-data-scientist-talla-at-mlconf-sf-2016
* [3] [DL輪読会] Hybrid computing using a neural network with dynamic external memory -  http://www.slideshare.net/YuusukeIwasawa/dl-hybrid-computing-using-a-neural-network-with-dynamic-external-memory
* [4] DNC review -  https://github.com/psygrammer/dgm/blob/master/part1/deepmind/DNC/Hybrid_computing_using_a_neural_network_with_dynamic_external_memory.ipynb

# Abstract

* Neural networks augmented with external memory have the ability to learn algorithmic solutions to complex tasks. 
    - These models appear promising for applications such as language modeling and machine translation. 
* <font color="blue">However, they scale poorly in both space and time as the amount of memory grows — limiting their applicability to real-world domains</font>. 
* <font color="red">Here, we present an end-to-end differentiable memory access scheme, which we call Sparse Access Memory (SAM)</font>, 
    - that retains the representational power of the original approaches whilst training efficiently with very large memories. 
    - We show that SAM achieves asymptotic lower bounds in space and time complexity, and find that an implementation runs <font color="blue">1,000× faster and with 3,000× less physical memory than non-sparse models</font>. 
    - SAM learns with comparable 
        - data efficiency to existing models on 
            - a range of synthetic tasks and 
            - one-shot Omniglot character recognition, and 
        - can scale to tasks requiring 100,000s of time steps and memories. 
* As well, we show how our approach can be 
    - <font color="red">adapted</font> for models that maintain temporal associations between memories, 
        - as with the recently introduced <font color="red">Differentiable Neural Computer</font>.


# 1 Introduction

* A significant difficulty in training these models results from their smooth read and write operations, which incur linear computational overhead on the number of memories stored per time step of training. Even worse, they require duplication of the entire memory at each time step to perform backpropagation through time (BPTT). To deal with sufficiently complex problems, such as processing book, or Wikipedia, this overhead becomes prohibitive. For example, to store 64 memories, a straightforward implementation of the NTM trained over a sequence of length 100 consumes ≈ 30 MiB physical memory; to store 64,000 memories the overhead exceeds 29 GiB (see Figure 1).

<img src="figures/cap6.png", width=600 />

* Further, in Supplementary D we demonstrate the generality of our approach by describing how to construct a sparse version of the recently published Differentiable Neural Computer [8]. This Sparse Differentiable Neural Computer (SDNC) is over 400× faster than the canonical dense variant for a memory size of 2,000 slots, and achieves the best reported result in the Babi tasks without supervising the memory access.

# 2 Background
* 2.1 Attention and content-based addressing
* 2.2 Memory Networks
* 2.3 Neural Turing Machine

## 2.1 Attention and content-based addressing

<img src="figures/cap1.png", width=600 />

## 2.2 Memory Networks

#### 참고
* [5] Neural networks with external memory - https://courses.cs.ut.ee/MTAT.03.292/2016_fall/uploads/Main/externalmemory.pdf

## 2.3 Neural Turing Machine

#### 참고
* [6] Neural Turing Machines - http://www.slideshare.net/iljakuzovkin/neural-turing-machines

The Neural Turing Machine is a recurrent neural network equipped with a content-addressable memory, similar to Memory Networks, but with the additional capability to write to memory over time. The memory is accessed by a controller network, typically an LSTM, and the full model is differentiable — allowing it to be trained via BPTT.

A write to memory,
<img src="figures/cap2.png", width=800 />

# 3 Architecture
* 3.1 Read
* 3.2 Write
* 3.3 Controller
* 3.4 Efficient backpropagation through time
* 3.5 Approximate nearest neighbors

This paper introduces Sparse Access Memory (SAM), a new neural memory architecture with two innovations. 
* Most importantly, 
    - all writes to and reads from external memory are 
        - constrained to a sparse subset of the memory words, 
            - providing similar functionality as the NTM, 
            - while allowing computational and memory efficient operation. 
* Secondly, 
    - we introduce a sparse memory management scheme that 
        - tracks memory usage and 
        - finds unused blocks of memory 
            - for recording new information.

For a memory containing N words, 
* SAM executes a forward, backward step in Θ(log N ) time, 
* initializes in Θ(N ) space, and 
* consumes Θ(1) space per time step. 
* Under some reasonable assumptions, 
    - SAM is asymptotically optimal in time and space complexity (Supplementary A).

## 3.1 Read

The sparse read operation is defined to be a weighted average over a selection of words in memory:
<img src="figures/cap3.png", width=800 />

O(N) time -> we can use an approximate nearest neighbor data-structure, described in Section 3.5, -> O(log N ) time

Sparse read can be considered a special case of the matrix-vector product defined in (1), with two key distinctions. 
* The first is that 
    - we pass gradients for 
        - only a constant K number of rows 
            - of memory per time step, versus N, 
        - which results in a negligible fraction of non-zero error gradient per timestep 
            - when the memory is large. 
* The second distinction is in implementation: 
    - by <font color="red">using an efficient sparse matrix format</font> such as <font color="blue">Compressed Sparse Rows (CSR)</font>, 
    - we can compute (4) and its gradients in constant time and space (see Supplementary A).

## 3.2 Write

* The write operation is SAM is an instance of (3) where the write weights w ̃tW are constrained to contain a constant number of non-zero entries. 
* This is done by a simple scheme where the controller writes either to previously read locations, in order to <font color="red">update contextually relevant memories</font>, or <font color="red">the least recently accessed location</font>, in order to overwrite stale or <font color="red">unused memory slots with fresh content</font>.
* The introduction of sparsity could be achieved via other write schemes. 
    - For example, we could use a sparse content-based write scheme, where the controller chooses a query vector qtW and applies writes to similar words in memory. This would allow for direct memory updates, but would create problems when the memory is empty (and shift further complexity to the controller). 
* We decided upon the <font color="red">previously read / least recently accessed addressing scheme</font> <font color="blue">for simplicity and flexibility</font>.

The write weights are defined as

<img src="figures/cap4.png", width=600 />

<img src="figures/cap5.png", width=800 />

## 3.3 Controller

We use a one layer LSTM for the controller throughout.

The LSTM then produces a vector, $p_t$ = ($q_t$, $a_t$, $α_t$, $γ_t$), of read and write parameters for memory access via a linear layer. 

<img src="figures/6fig.png" width=600 />

## 3.4 Efficient backpropagation through time

<img src="figures/fig5.png" width=800 />

## 3.5 Approximate nearest neighbors

When querying the memory, 
* we can use an approximate nearest neighbor index (ANN) to search over the external memory for the K nearest words. 
* Where a linear KNN search inspects every element in memory (taking O(N) time), an ANN index maintains a structure over the dataset to allow for fast inspection of nearby points in O(log N ) time.

We considered two types of ANN indexes: 
* FLANN’s randomized k-d tree implementation that 
    - arranges the datapoints in an ensemble of structured (randomized k-d) trees to search for nearby points via comparison-based search, and 
* one that uses locality sensitive hash (LSH) functions that 
    - map points into buckets with distance-preserving guarantees. 
    
We used randomized k-d trees for small word sizes and LSHs for large word sizes. 
* For both ANN implementations, there is an O(log N ) cost for insertion, deletion and query.

We also rebuild the ANN from scratch every N insertions to ensure it does not become imbalanced.

# 4 Results
* 4.1 Speed and memory benchmarks
* 4.2 Learning with sparse memory access
* 4.3 Scaling with a curriculum
* 4.4 Question answering on the Babi tasks
* 4.5 Learning on real world data

## 4.1 Speed and memory benchmarks

<img src="figures/cap6.png", width=600 />

## 4.2 Learning with sparse memory access

<img src="figures/cap7.png", width=600 />

## 4.3 Scaling with a curriculum

<img src="figures/cap8.png", width=600 />

<img src="figures/fig8.png", width=600 />

## 4.4 Question answering on the Babi tasks

#### 참고
* [7] Playground for bAbI tasks - https://yerevann.github.io/2016/02/23/playground-for-babi-tasks/
* [8] Dynamic Memory Networks by YerevanNN Web Demo - http://yerevann.com/dmn-ui/#/

<img src="figures/tab1.png" width=800 />

## 4.5 Learning on real world data

#### 참고
* [9] brendenlake/omniglot - https://github.com/brendenlake/omniglot
* [10] One-Shot Learning - http://www.slideshare.net/JisungDavidKim/oneshot-learning

<img src="figures/cap9.png", width=600 />

# 5 Discussion

# 참고자료
* [1] Scaling Memory-Augmented Neural Networks with Sparse Reads and Writes - https://arxiv.org/abs/1610.09027
* [2] Daniel Shank, Data Scientist, Talla at MLconf SF 2016 - http://www.slideshare.net/SessionsEvents/daniel-shank-data-scientist-talla-at-mlconf-sf-2016
* [3] [DL輪読会] Hybrid computing using a neural network with dynamic external memory -  http://www.slideshare.net/YuusukeIwasawa/dl-hybrid-computing-using-a-neural-network-with-dynamic-external-memory
* [4] DNC review -  https://github.com/psygrammer/dgm/blob/master/part1/deepmind/DNC/Hybrid_computing_using_a_neural_network_with_dynamic_external_memory.ipynb
* [5] Neural networks with external memory - https://courses.cs.ut.ee/MTAT.03.292/2016_fall/uploads/Main/externalmemory.pdf
* [6] Neural Turing Machines - http://www.slideshare.net/iljakuzovkin/neural-turing-machines
* [7] Playground for bAbI tasks - https://yerevann.github.io/2016/02/23/playground-for-babi-tasks/
* [8] Dynamic Memory Networks by YerevanNN Web Demo - http://yerevann.com/dmn-ui/#/
* [9] brendenlake/omniglot - https://github.com/brendenlake/omniglot
* [10] One-Shot Learning - http://www.slideshare.net/JisungDavidKim/oneshot-learning